## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-25-20-53-29 +0000,nyt,Deputy attorney general takes a measured tone ...,https://www.nytimes.com/live/2026/01/25/us/min...
1,2026-01-25-20-52-17 +0000,nyt,"While many churches canceled Sunday services, ...",https://www.nytimes.com/live/2026/01/25/us/win...
2,2026-01-25-20-52-00 +0000,wsj,Expansive Winter Storm Snarls the Central and ...,https://www.wsj.com/us-news/climate-environmen...
3,2026-01-25-20-51-39 +0000,nypost,NJ man ‘sipping coffee’ as $5k robotic slow bl...,https://nypost.com/2026/01/25/us-news/nj-man-s...
4,2026-01-25-20-50-00 +0000,wsj,Opinion | Ro and Our New Health Sherpas,https://www.wsj.com/opinion/ro-and-our-new-hea...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2433/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,shooting,28
6,trump,27
118,minneapolis,22
81,alex,18
82,pretti,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2026-01-25-18-53-33 +0000,nypost,Barack Obama issues rare political statement c...,https://nypost.com/2026/01/25/us-news/barack-o...,115
58,2026-01-25-16-33-57 +0000,nypost,Trump-appointee judge orders DHS to preserve e...,https://nypost.com/2026/01/25/us-news/trump-ap...,112
115,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,107
76,2026-01-25-15-31-54 +0000,bbc,Video shows moments around fatal shooting of A...,https://www.bbc.com/news/videos/cp372pqq2rlo?a...,105
57,2026-01-25-16-46-09 +0000,nyt,"In Court Filings, Witnesses Describe Fatal Min...",https://www.nytimes.com/2026/01/24/us/witnesse...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,115,2026-01-25-18-53-33 +0000,nypost,Barack Obama issues rare political statement c...,https://nypost.com/2026/01/25/us-news/barack-o...
184,57,2026-01-24-21-35-27 +0000,nypost,AOC urges US Senate to block ICE funding after...,https://nypost.com/2026/01/24/us-news/aoc-urge...
64,46,2026-01-25-16-22-26 +0000,nypost,Here’s what airlines owe you if you’re strande...,https://nypost.com/2026/01/25/us-news/heres-wh...
40,34,2026-01-25-18-44-00 +0000,wsj,"In Battle for Evidence, Minnesota Sues Feds Ov...",https://www.wsj.com/us-news/in-battle-for-evid...
73,33,2026-01-25-15-41-03 +0000,cbc,"No plans for China free trade deal, Carney say...",https://www.cbc.ca/news/politics/carney-canada...
48,30,2026-01-25-17-55-08 +0000,nyt,New York City Schools Announce Remote Learning...,https://www.nytimes.com/2026/01/25/us/nyc-scho...
134,29,2026-01-25-06-02-10 +0000,nypost,Florida Rep. Maxwell Frost says he was punched...,https://nypost.com/2026/01/25/us-news/florida-...
56,25,2026-01-25-17-00-00 +0000,wsj,U.S. Speaker of the House Mike Johnson has hel...,https://www.wsj.com/politics/republicans-house...
142,24,2026-01-25-04-44-43 +0000,nypost,"Daredevil scales 1,667-foot skyscraper without...",https://nypost.com/2026/01/24/lifestyle/rock-c...
180,22,2026-01-24-22-08-28 +0000,nypost,"US envoys Steve Witkoff, Jared Kushner meet wi...",https://nypost.com/2026/01/24/world-news/us-en...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
